<a href="https://colab.research.google.com/github/OFCMatt/EBAC/blob/main/Profiss%C3%A3o_Cientista_de_Dados_M4_Tarefa_aulas_06_a_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Consultar a API para municípios do RJ
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
params = {
    "uf": "RJ",
    "interessado": "Município"
}
response = requests.get(url, params=params)
dados = pd.DataFrame(response.json()['items'])

In [3]:
# 2) Ver os 3 status mais frequentes
status_freq = dados['status'].value_counts().head(3)
print("Top 3 status mais frequentes:")
print(status_freq)

Top 3 status mais frequentes:
status
Deferido                                                 120
Arquivado                                                 53
Encaminhado à PGFN com manifestação técnica favorável     50
Name: count, dtype: int64


In [4]:
# 3) Criar coluna com o ano do status
dados['ano_status'] = dados['data_status'].str.slice(0, 4)

In [5]:
# 4) Frequência de cada ano
frequencia_anos = dados['ano_status'].value_counts()
print("Frequência por ano do status:")
print(frequencia_anos)

Frequência por ano do status:
ano_status
13/0    16
28/0    16
06/0    15
29/0    14
27/0    13
15/0    13
26/0    12
22/0    10
04/0    10
30/1    10
12/0    10
30/0     9
18/0     9
10/0     9
25/0     9
11/0     8
02/0     7
20/0     7
01/0     7
04/1     6
14/0     6
03/0     6
21/0     6
23/0     5
17/0     5
23/1     5
21/1     5
07/0     5
09/0     5
10/1     4
01/1     4
31/0     4
28/1     4
24/0     4
05/0     3
17/1     3
19/1     3
19/0     3
27/1     3
22/1     3
06/1     2
13/1     2
08/0     2
20/1     2
18/1     2
02/1     2
16/0     2
09/1     2
25/1     2
31/1     1
07/1     1
29/1     1
14/1     1
15/1     1
24/1     1
26/1     1
08/1     1
11/1     1
12/1     1
05/1     1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [6]:
def consulta_pvl(uf, interessado):
    url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    params = {
        "uf": uf,
        "interessado": interessado
    }
    response = requests.get(url, params=params)
    dados = pd.DataFrame(response.json()['items'])
    return dados


In [7]:
# Consultar dados de MG com interessado = Estado
mg_estado = consulta_pvl("MG", "Estado")

# Filtrar por status
arquivados = mg_estado[mg_estado["status"] == "Arquivado por decurso de prazo"]
print("Quantidade de solicitações arquivadas por decurso de prazo em MG:", len(arquivados))


Quantidade de solicitações arquivadas por decurso de prazo em MG: 306


In [8]:
# Consultar dados de municípios da BA
ba_municipios = consulta_pvl("BA", "Município")

# Filtrar apenas os deferidos
deferidos = ba_municipios[ba_municipios["status"] == "Deferido"]

# Contar por município
mais_deferido = deferidos["interessado"].value_counts().idxmax()
qtd_deferido = deferidos["interessado"].value_counts().max()

print(f"Município da BA com mais deferimentos: {mais_deferido} ({qtd_deferido} solicitações)")


Município da BA com mais deferimentos: Bahia (50 solicitações)


In [9]:
# Consultar dados da Bahia - Estado
ba_estado = consulta_pvl("BA", "Estado")

# Salvar em CSV
ba_estado.to_csv("solicitacoes_bahia_estado.csv", index=False, encoding='utf-8')
print("Arquivo salvo: solicitacoes_bahia_estado.csv")


Arquivo salvo: solicitacoes_bahia_estado.csv
